In [ ]:
from skimage.color import rgb2lab, deltaE_ciede2000
from skimage import io
import numpy as np
import skimage
from math import ceil
import threading


## Define the pallette 

In [ ]:
# Nord Aurora theme
# pallette_hex=[
# # Snow Storm
# "#D8DEE9",
# "#E5E9F0",
# "#ECEFF4",
# #polar Night
# "#2E3440",
# "#3B4252",
# "#434C5E",
# "#4C566A",
# # Frost 
# "#8FBCBB",
# "#88C0D0",
# "#81A1C1",
# "#5E81AC",
# # Aurora
# "#BF616A",
# "#D08770",
# "#EBCB8B",
# "#A3BE8C",
# "#B48EAD",
# ]
# Amarena theme
pallette_hex=[
"#242D35",
"#FB6396",
"#94CF95",
"#F692B2",
"#6EC1D6",
"#CD84C8",
"#7FE4D2",
"#FFFFFF",
"#526170",
"#F92D72",
"#6CCB6E",
"#F26190",
"#4CB9D6",
"#C269BC",
"#58D6BF",
"#F4F5F2",
]
pallette_rgb=[]
for h in pallette_hex:
    temp=h.lstrip('#')
    pallette_rgb.append(tuple(int(temp[i:i+2], 16) for i in (0, 2 ,4)))
pallette_rgb=np.asarray(pallette_rgb,dtype=np.uint8)
pallette=np.array([rgb2lab(pallette_rgb[i]) for i in range(len(pallette_rgb))])
print(pallette_rgb)

## Some Utility Functions

In [ ]:
def convert_rgb_to_lab(col):
    # print("Rgb: ",col)
    temp=col[:3]
    im=np.asarray(temp,dtype=np.uint8)
    cov=rgb2lab(im)
    # print("LAB: ",cov)
    return cov


In [ ]:
def color_diff(color1,color2):
    delta_e=deltaE_ciede2000(color1,color2)
    # print("DeltaE: ",delta_e)
    return delta_e   

## Open the image file

Open the image file and convert it to LAB space

In [ ]:
img=io.imread("astronaut-jellyfish.jpg")
isrgba=False
img2=img.copy()
if(img[0][0].size==4):
    isrgba=True
    img=skimage.color.rgba2rgb(img)
og_img_lab=rgb2lab(img)

## Themify

for every pixel find the minimum distance between the pixel and the colors in the palette in LAB colorspace. Choose the palette color which corresponds to the minimum distance.

Store the `pixel_color->palette_color pair` in a hash map to prevent redundant recalculation.

In [ ]:
pix_theme_map={}
def themify_img_v2(og_img,img,start_row,end_row,start_col,end_col):
    for i in range(start_row,min(end_row,img.shape[0])):
        for j in range(start_col,min(end_col,img.shape[1])):
            key=np.array2string(og_img[i][j],separator=',')
            if key in pix_theme_map:
                img[i][j]=pix_theme_map[key]
                continue
            # METHOD-1: 
            # target=pallette[0]
            # diff=100000
            # for p in pallette:
            #     temp=color_diff(p, og_img[i][j])
            #     if temp < diff:
            #         diff=temp
            #         target=p
            
            # print(lab2rgb(target))
            # col=lab2rgb(target);
            # col[0]*=255;col[1]*=255;col[2]*=255;
            # img[i][j]=[*col,img[i][j][3]]
            # pix_theme_map[key]=[*col,img[i][j][3]]
            
            # METHOD-2:
            target=0;
            diff=color_diff(pallette[0], og_img[i][j])
            for k in range(1,len(pallette)):
                temp=color_diff(pallette[k], og_img[i][j])
                if temp < diff:
                    diff=temp
                    target=k
            if isrgba:
                img[i][j]=[*pallette_rgb[target],img[i][j][3]]
                pix_theme_map[key]=[*pallette_rgb[target],img[i][j][3]]
            else:
                img[i][j]=pallette_rgb[target]
                pix_theme_map[key]=pallette_rgb[target]
            

## Multithreading

Use multithreading to reduce time taken. With 40 threads it takes around 10 mins for large images.

In [ ]:
MAX_THREADS=40
step=ceil(img.shape[0]/MAX_THREADS)
print(step)
threads=[]
for row in range(step,img.shape[0]+step,step):
    args=(og_img_lab,img2, row-step,row+1,0,img.shape[1])
    t=threading.Thread(target=themify_img_v2,args=args)
    t.daemon = True
    t.start()
    threads.append(t)
print("finished threads")
for t in threads:
    t.join(timeout=60)


### Save the image

In [ ]:
io.imsave("temp.png",img2)


In [ ]:
# img3=skimage.filters.gaussian(img2,sigma=14,mode="reflect")
# io.imsave("blur-temp.png",img3)